# Making models using Default config

## ResNet

In [1]:
from detectron2.config import get_cfg
from detectron2.modeling import build_resnet_backbone
from detectron2.layers import ShapeSpec
from torchinfo import summary

# loads default config
cfg = get_cfg()

default_resnet_backbone = build_resnet_backbone(cfg=cfg, input_shape=ShapeSpec(channels=3))

print(summary(default_resnet_backbone))
type(default_resnet_backbone)

Layer (type:depth-idx)                        Param #
ResNet                                        --
├─BasicStem: 1-1                              --
│    └─Conv2d: 2-1                            --
│    │    └─FrozenBatchNorm2d: 3-1            --
├─Sequential: 1-2                             --
│    └─BottleneckBlock: 2-2                   --
│    │    └─Conv2d: 3-2                       (16,384)
│    │    └─Conv2d: 3-3                       (4,096)
│    │    └─Conv2d: 3-4                       (36,864)
│    │    └─Conv2d: 3-5                       (16,384)
│    └─BottleneckBlock: 2-3                   --
│    │    └─Conv2d: 3-6                       (16,384)
│    │    └─Conv2d: 3-7                       (36,864)
│    │    └─Conv2d: 3-8                       (16,384)
│    └─BottleneckBlock: 2-4                   --
│    │    └─Conv2d: 3-9                       (16,384)
│    │    └─Conv2d: 3-10                      (36,864)
│    │    └─Conv2d: 3-11                      (16,384)
├─Seq

detectron2.modeling.backbone.resnet.ResNet

In [2]:
import torch

dummy_in = torch.zeros((1, 3, 256, 256))
default_resnet_backbone_out = default_resnet_backbone(dummy_in)  # returns a dict[str -> torch.Tensor]

# only res4 here because specified in default config
for k, v in default_resnet_backbone_out.items():
    print(k, type(v), v.shape)

res4 <class 'torch.Tensor'> torch.Size([1, 1024, 16, 16])


## FPN

In [3]:
from detectron2.modeling.backbone.fpn import build_resnet_fpn_backbone

cfg = get_cfg()

# make changes to the default config
# for some reason(which i don't understand yet) it must contain "res5", ie features after last stage
cfg.MODEL.RESNETS.OUT_FEATURES = ["res3", "res4", "res5"]
cfg.MODEL.FPN.IN_FEATURES = ["res3", "res4", "res5"]

default_fpn_backbone = build_resnet_fpn_backbone(cfg=cfg, input_shape=ShapeSpec(channels=3))

type(default_fpn_backbone)

detectron2.modeling.backbone.fpn.FPN

In [4]:
dummy_in = torch.zeros((1, 3, 256, 256))
default_fpn_backbone_out = default_fpn_backbone(dummy_in)

for k,v in default_fpn_backbone_out.items():
    print(k, type(v), v.shape)

p3 <class 'torch.Tensor'> torch.Size([1, 256, 32, 32])
p4 <class 'torch.Tensor'> torch.Size([1, 256, 16, 16])
p5 <class 'torch.Tensor'> torch.Size([1, 256, 8, 8])
p6 <class 'torch.Tensor'> torch.Size([1, 256, 4, 4])


# Making models using Custom config
Use detectron's implementation of fpn and resnet to make our own

## Saving and loading a config

In [5]:
# first let's save default config to disk
import os
from pathlib import Path

default_config = get_cfg()

configs_dir = os.path.join(Path(os.path.abspath('')).parent, 'configs')
default_config_fname = os.path.join(configs_dir, 'detectron_default.yaml')

with open(default_config_fname, 'w') as f:
    f.write(default_config.dump())

In [6]:
# load back config from disk
from detectron2.config import CfgNode

# loads the config as a dict
default_config_dict = CfgNode.load_yaml_with_base(filename=default_config_fname)
print(type(default_config_dict))

# convert dict to CfgNode
default_config_loaded = CfgNode(init_dict=default_config_dict)
print(type(default_config_loaded))

<class 'dict'>
<class 'detectron2.config.config.CfgNode'>


In [7]:
# make a func to load a config given filename
def load_config(filename: str):
    config_dict = CfgNode.load_yaml_with_base(filename=filename)
    return CfgNode(init_dict=config_dict)


type(load_config(default_config_fname))

detectron2.config.config.CfgNode

## ResNet using custom config

In [8]:
from detectron2.modeling.backbone.resnet import BasicBlock, BasicStem, ResNet
from torch import nn

resnet_custom_config_fname = os.path.join(Path(os.path.abspath('')).parent, 'configs', 'resnet.yaml')
resnet_config = load_config(resnet_custom_config_fname)

# this stem only supports R18/R34 for now
def build_resnet(cfg):
    """Create a resnet instance from config """

    resnet_stem = BasicStem()

    depth = cfg.MODEL.RESNET.DEPTH
    out_features = cfg.MODEL.RESNET.OUT_FEATURES

    in_channels = cfg.MODEL.RESNET.STEM_OUT_CHANNELS
    out_channels = cfg.MODEL.RESNET.RES2_OUT_CHANNELS

    num_blocks_per_stage = {
        18: [2, 2, 2, 2],
        34: [3, 4, 6, 3],
        50: [3, 4, 6, 3],
        101: [3, 4, 23, 3],
        152: [3, 8, 36, 3],
    }[depth]

    assert out_channels == 64, "Must set MODELS.RESNET.RES2_OUT_CHANNELS = 64 for R18/R34"

    stages = []

    for idx, stage_idx in enumerate(range(2,  6)):
        first_stride = 1 if idx == 0 else 2
        stage_kargs = {
            "num_blocks": num_blocks_per_stage[idx],
            "stride_per_block": [first_stride] + [1] * (num_blocks_per_stage[idx] - 1),
            "in_channels": in_channels,
            "out_channels": out_channels
        }

        if depth in {18, 34}:  # always true for now, changes for bigger models
            stage_kargs['block_class'] = BasicBlock

        blocks = ResNet.make_stage(**stage_kargs)

        in_channels = out_channels
        out_channels *= 2

        stages.append(blocks)

    CNN = ResNet(
        stem=resnet_stem,
        stages=stages,
        out_features=out_features
    )

    return CNN


resnet_backbone = build_resnet(cfg=resnet_config)
type(resnet_backbone)

detectron2.modeling.backbone.resnet.ResNet

In [9]:
from torchinfo import summary

print(summary(resnet_backbone, input_size=(1, 3, 256, 256), col_names=['output_size', 'num_params', 'kernel_size']))

dummy_in = torch.zeros((1, 3, 256, 256))
dummy_out = resnet_backbone(dummy_in)

for k, v in dummy_out.items():
    print(k, type(v), v.shape)

Layer (type:depth-idx)                   Output Shape              Param #                   Kernel Shape
ResNet                                   --                        --                        --
├─BasicStem: 1-1                         [1, 64, 64, 64]           --                        --
│    └─Conv2d: 2-1                       [1, 64, 128, 128]         --                        [3, 64, 7, 7]
│    │    └─BatchNorm2d: 3-1             [1, 64, 128, 128]         128                       [64]
├─Sequential: 1-2                        [1, 64, 64, 64]           --                        --
│    └─BasicBlock: 2-2                   [1, 64, 64, 64]           --                        --
│    │    └─Conv2d: 3-2                  [1, 64, 64, 64]           36,992                    [64, 64, 3, 3]
│    │    └─Conv2d: 3-3                  [1, 64, 64, 64]           36,992                    [64, 64, 3, 3]
│    └─BasicBlock: 2-3                   [1, 64, 64, 64]           --                    

## Join Resnet backbone with branches

In [10]:
def build_bottom_up(backbone):
    # dummy forward function of the stem
    dummy_in = torch.zeros((1, 3, 256, 256))
    backbone_cnn_features = backbone(dummy_in)

    adaptive_pool_layer = nn.AdaptiveAvgPool2d((1, 1))
    backbone_out = torch.flatten(adaptive_pool_layer(backbone_cnn_features['res5']), 1)
    print(backbone_out.size())

    branch1 = nn.Linear(in_features=512, out_features=10)
    branch2 = nn.Linear(in_features=512, out_features=10)
    branch3 = nn.Linear(in_features=512, out_features=10)

    branch1_out = branch1(backbone_out)
    branch2_out = branch2(backbone_out)
    branch3_out = branch3(backbone_out)

    print(branch1_out.size(), branch2_out.size(), branch3_out.size())


build_bottom_up(backbone=resnet_backbone)

torch.Size([1, 512])
torch.Size([1, 10]) torch.Size([1, 10]) torch.Size([1, 10])


## Build FPN using our resnet backbone

In [12]:
from detectron2.modeling.backbone.fpn import FPN, LastLevelMaxPool

def build_fpn(cfg):
    resnet_backbone = build_resnet(cfg)

    in_features = cfg.MODEL.FPN.IN_FEATURES
    out_channels = cfg.MODEL.FPN.OUT_CHANNELS

    fpn = FPN(
        bottom_up=resnet_backbone,
        in_features=in_features,
        out_channels=out_channels,
        top_block=LastLevelMaxPool()
    )

    return fpn


resnet_config = load_config(resnet_custom_config_fname)
my_fpn = build_fpn(resnet_config)
type(my_fpn)

detectron2.modeling.backbone.fpn.FPN

In [13]:
dummy_in = torch.zeros((1, 3, 256, 256))
dummy_out = my_fpn(dummy_in)

for k, v in dummy_out.items():
    print(k, type(v), v.shape)

p4 <class 'torch.Tensor'> torch.Size([1, 128, 16, 16])
p5 <class 'torch.Tensor'> torch.Size([1, 128, 8, 8])
p6 <class 'torch.Tensor'> torch.Size([1, 128, 4, 4])
